In [1]:
import time, csv
import pandas as pd
from pyais import decode, IterMessages
from pyais.stream import FileReaderStream
from pyais.messages import MessageType1
from  collections import defaultdict as ddict
from pyais.stream import TCPConnection
from contextlib import ExitStack

In [2]:
decoded = decode(b"!AIVDM,1,1,,B,15NG6V0P01G?cFhE`R2IU?wn28R>,0*05")
print(decoded)

MessageType1(msg_type=1, repeat=0, mmsi=367380120, status=<NavigationStatus.UnderWayUsingEngine: 0>, turn=<TurnRate.NO_TI_DEFAULT: -128.0>, speed=0.1, accuracy=False, lon=-122.404333, lat=37.806948, course=245.2, heading=511, second=59, maneuver=<ManeuverIndicator.NotAvailable: 0>, spare_1=b'\x00', raim=True, radio=34958)


In [3]:
parts = [
    b"!AIVDM,2,1,4,A,55O0W7`00001L@gCWGA2uItLth@DqtL5@F22220j1h742t0Ht0000000,0*08",
    b"!AIVDM,2,2,4,A,000000000000000,2*20",
]

# Decode a multipart message using decode
decoded = decode(*parts)
as_dict = decoded.asdict()
print(as_dict)

{'msg_type': 5, 'repeat': 0, 'mmsi': 368060190, 'ais_version': 2, 'imo': 0, 'callsign': 'WDK4954', 'shipname': 'P/V_GOLDEN_GATE', 'ship_type': <ShipType.PilotVessel: 50>, 'to_bow': 14, 'to_stern': 7, 'to_port': 4, 'to_starboard': 2, 'epfd': <EpfdType.Internal_GNSS: 15>, 'month': 0, 'day': 0, 'hour': 24, 'minute': 60, 'draught': 0.0, 'destination': '', 'dte': False, 'spare_1': b'\x00'}


In [4]:
fake_stream = [
    b"!AIVDM,1,1,,A,13HOI:0P0000VOHLCnHQKwvL05Ip,0*23",
    b"!AIVDM,1,1,,A,133sVfPP00PD>hRMDH@jNOvN20S8,0*7F",
    b"!AIVDM,1,1,,B,100h00PP0@PHFV`Mg5gTH?vNPUIp,0*3B",
    b"!AIVDM,2,1,4,A,55O0W7`00001L@gCWGA2uItLth@DqtL5@F22220j1h742t0Ht0000000,0*08",
    b"!AIVDM,2,2,4,A,000000000000000,2*20",
    b"!AIVDM,1,1,,B,13eaJF0P00Qd388Eew6aagvH85Ip,0*45",
    b"!AIVDM,1,1,,A,14eGrSPP00ncMJTO5C6aBwvP2D0?,0*7A",
    b"!AIVDM,1,1,,A,15MrVH0000KH<:V:NtBLoqFP2H9:,0*2F",
]
for i, msg in enumerate(IterMessages(fake_stream)):
    print(i)
    print(msg.decode())

0
MessageType1(msg_type=1, repeat=0, mmsi=227006760, status=<NavigationStatus.UnderWayUsingEngine: 0>, turn=<TurnRate.NO_TI_DEFAULT: -128.0>, speed=0.0, accuracy=False, lon=0.13138, lat=49.475577, course=36.7, heading=511, second=14, maneuver=<ManeuverIndicator.NotAvailable: 0>, spare_1=b'\x00', raim=False, radio=22136)
1
MessageType1(msg_type=1, repeat=0, mmsi=205448890, status=<NavigationStatus.UnderWayUsingEngine: 0>, turn=<TurnRate.NO_TI_DEFAULT: -128.0>, speed=0.0, accuracy=True, lon=4.419442, lat=51.237658, course=63.3, heading=511, second=15, maneuver=<ManeuverIndicator.NotAvailable: 0>, spare_1=b'\x00', raim=True, radio=2248)
2
MessageType1(msg_type=1, repeat=0, mmsi=786434, status=<NavigationStatus.UnderWayUsingEngine: 0>, turn=<TurnRate.NO_TI_DEFAULT: -128.0>, speed=1.6, accuracy=True, lon=5.320033, lat=51.967037, course=112.0, heading=511, second=15, maneuver=<ManeuverIndicator.NoSpecialManeuver: 1>, spare_1=b'\x00', raim=False, radio=153208)
3
MessageType5(msg_type=5, repea

# Read from a test file:

In [5]:
filepath = "data/nmea_data_sample.txt"

found = ddict(bool)
i = 0
with FileReaderStream(filepath) as stream:
    for msg in stream:
        decoded = msg.decode()
        if not found[decoded.msg_type]:
            print("msg type: ",decoded.asdict()['msg_type'], list(decoded.asdict().keys()))
            found[decoded.msg_type] = True
        if i == 1000:
            break
        i += 1 

msg type:  3 ['msg_type', 'repeat', 'mmsi', 'status', 'turn', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'second', 'maneuver', 'spare_1', 'raim', 'radio']
msg type:  1 ['msg_type', 'repeat', 'mmsi', 'status', 'turn', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'second', 'maneuver', 'spare_1', 'raim', 'radio']
msg type:  18 ['msg_type', 'repeat', 'mmsi', 'reserved_1', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'second', 'reserved_2', 'cs', 'display', 'dsc', 'band', 'msg22', 'assigned', 'raim', 'radio']
msg type:  4 ['msg_type', 'repeat', 'mmsi', 'year', 'month', 'day', 'hour', 'minute', 'second', 'accuracy', 'lon', 'lat', 'epfd', 'spare_1', 'raim', 'radio']
msg type:  15 ['msg_type', 'repeat', 'mmsi', 'spare_1', 'mmsi1', 'type1_1', 'offset1_1', 'spare_2', 'type1_2', 'offset1_2', 'spare_3', 'mmsi2', 'type2_1', 'offset2_1', 'spare_4']
msg type:  24 ['msg_type', 'repeat', 'mmsi', 'partno', 'shipname', 'spare_1']
msg type:  20 ['msg_type', 'repeat', 'mm

# Testing Live AIS feed from the Norvegian Coastal Administration

In [20]:
host = '153.44.253.27'
port = 5631
start = time.time()

for msg in TCPConnection(host, port=port):
    decoded_message = msg.decode()
    ais_content = decoded_message

    print('*' * 80)
    if msg.tag_block:
        # decode & print the tag block if it is available
        msg.tag_block.init()
        print(msg.tag_block.asdict())

    print(ais_content)
    if time.time() > start + 20:  # Print the feed for 20 second
        break

********************************************************************************
{'raw': b's:2573335,c:1732120732*08', 'receiver_timestamp': '1732120732', 'source_station': '2573335', 'destination_station': None, 'line_count': None, 'relative_time': None, 'text': None}
MessageType24PartA(msg_type=24, repeat=0, mmsi=258019240, partno=0, shipname='FROYBRAND', spare_1=None)
********************************************************************************
{'raw': b's:2573415,c:1732120732*0D', 'receiver_timestamp': '1732120732', 'source_station': '2573415', 'destination_station': None, 'line_count': None, 'relative_time': None, 'text': None}
MessageType1(msg_type=1, repeat=0, mmsi=257714500, status=<NavigationStatus.UnderWayUsingEngine: 0>, turn=0.0, speed=8.6, accuracy=True, lon=13.12606, lat=66.67426, course=14.1, heading=26, second=50, maneuver=<ManeuverIndicator.NotAvailable: 0>, spare_1=b'\x00', raim=False, radio=34751)
*******************************************************************

# Inspection of the norvegian feed and determine the fields for the messages types
Watch out the feed goes forever if not stop with keyboard. 

In [7]:
first = ddict(bool)
host = '153.44.253.27'
port = 5631
start = time.time()
try:
    for msg in TCPConnection(host, port=port):
        decoded_message = msg.decode()
        ais_content = decoded_message
        ais_content = ais_content.asdict()

        if msg.tag_block:
            # decode & print the tag block if it is available
            msg.tag_block.init()
            # writer.writerow({'first_name': 'Wonderful', 'last_name': 'Spam'})
            tag_block = msg.tag_block.asdict()
            ais_content['receiver_timestamp'] = tag_block['receiver_timestamp']
        if not first[ais_content['msg_type']]:
            print(ais_content['msg_type'], list(ais_content.keys()))
            first[ais_content['msg_type']] = True
except KeyboardInterrupt:
    pass

24 ['msg_type', 'repeat', 'mmsi', 'partno', 'shipname', 'spare_1', 'receiver_timestamp']
3 ['msg_type', 'repeat', 'mmsi', 'status', 'turn', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'second', 'maneuver', 'spare_1', 'raim', 'radio', 'receiver_timestamp']
5 ['msg_type', 'repeat', 'mmsi', 'ais_version', 'imo', 'callsign', 'shipname', 'ship_type', 'to_bow', 'to_stern', 'to_port', 'to_starboard', 'epfd', 'month', 'day', 'hour', 'minute', 'draught', 'destination', 'dte', 'spare_1', 'receiver_timestamp']
1 ['msg_type', 'repeat', 'mmsi', 'status', 'turn', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'second', 'maneuver', 'spare_1', 'raim', 'radio', 'receiver_timestamp']
8 ['msg_type', 'repeat', 'mmsi', 'spare_1', 'dac', 'fid', 'data', 'receiver_timestamp']
18 ['msg_type', 'repeat', 'mmsi', 'reserved_1', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'second', 'reserved_2', 'cs', 'display', 'dsc', 'band', 'msg22', 'assigned', 'raim', 'radio', 'receiver_timestam

# With the previous message field description, we save 20 minute of the norvegian coast guard feed.

In [22]:
recording_time = 20  # in minute
msg_type = [24, 3, 1, 5, 21, 18, 19, 8]
first = ddict(bool)
msg_columns = {1 : ['msg_type', 'receiver_timestamp', 'repeat', 'mmsi', 'status', 'turn', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'second', 'maneuver', 'spare_1', 'raim', 'radio'],
               3 : ['msg_type', 'receiver_timestamp', 'repeat', 'mmsi', 'status', 'turn', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'second', 'maneuver', 'spare_1', 'raim', 'radio'],
               5 : ['msg_type', 'receiver_timestamp', 'repeat', 'mmsi', 'ais_version', 'imo', 'callsign', 'shipname', 'ship_type', 'to_bow', 'to_stern', 'to_port', 'to_starboard', 'epfd', 'month', 'day', 'hour', 'minute', 'draught', 'destination', 'dte', 'spare_1'],
               8 : ['msg_type', 'receiver_timestamp', 'repeat', 'mmsi', 'spare_1', 'dac', 'fid', 'data'],
               18 : ['msg_type', 'receiver_timestamp', 'repeat', 'mmsi', 'reserved_1', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'second', 'reserved_2', 'cs', 'display', 'dsc', 'band', 'msg22', 'assigned', 'raim', 'radio'],
               19 : ['msg_type', 'receiver_timestamp', 'repeat', 'mmsi', 'reserved_1', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'second', 'reserved_2', 'shipname', 'ship_type', 'to_bow', 'to_stern', 'to_port', 'to_starboard', 'epfd', 'raim', 'dte', 'assigned', 'spare_1'],
               21 : ['msg_type', 'receiver_timestamp', 'repeat', 'mmsi', 'aid_type', 'name', 'accuracy', 'lon', 'lat', 'to_bow', 'to_stern', 'to_port', 'to_starboard', 'epfd', 'second', 'off_position', 'reserved_1', 'raim', 'virtual_aid', 'assigned', 'spare_1', 'name_ext'],
               24 : ['msg_type', 'receiver_timestamp', 'repeat', 'mmsi', 'partno', 'shipname', 'spare_1', 'to_starboard', 'callsign', 'serial', 'ship_type', 'model', 'vendorid', 'to_port', 'to_bow', 'to_stern']}

with ExitStack() as stack:   # allows for multiple with open statement on two lines
    # open n files for each message type: 1, 3, 5, 8, 18, 19, 21 et 24
    files = {n : stack.enter_context(open(f"data/norvegian_20_min/message_{n}.csv", 'w', newline='')) for n in msg_type}
    csv_writers = {n : csv.DictWriter(file, msg_columns[n]) for n, file in files.items()}
    
    host = '153.44.253.27'
    port = 5631
    start = time.time()
    
    for msg in TCPConnection(host, port=port):
        decoded_message = msg.decode()
        ais_content = decoded_message
        ais_content = ais_content.asdict()

        if msg.tag_block:
            msg.tag_block.init()
            tag_block = msg.tag_block.asdict()
            ais_content['receiver_timestamp'] = tag_block['receiver_timestamp']

        writer = csv_writers[ais_content['msg_type']]   # select csv_writer to file that correspond to the msg_type
        if not first[ais_content['msg_type']]:   # If first message of that msg_type, then write the header of the file.
            writer.writeheader()
            first[ais_content['msg_type']] = True
        try:
            writer.writerow(ais_content)
        except ValueError as e:
            print(e, f"    msg_type: {ais_content['msg_type']}")
            break

        if time.time() > start + (recording_time * 60):   # stop recording after N minutes.
            break
    